In [26]:
!gdown 13oHjgTHTFc06u0gw0A-P4wq5G87Ni6nE
!mkdir ./raw_dataset/
!unzip vk_pokemons.zip
!mv ./images/ ./raw_dataset/
!rm vk_pokemons.zip

Downloading...
From (original): https://drive.google.com/uc?id=13oHjgTHTFc06u0gw0A-P4wq5G87Ni6nE
From (redirected): https://drive.google.com/uc?id=13oHjgTHTFc06u0gw0A-P4wq5G87Ni6nE&confirm=t&uuid=09c00f4b-4a83-4e51-a957-17fc05924f09
To: /Users/naer/work/SD-pokemons/data/vk_pokemons.zip
100%|██████████████████████████████████████| 60.7M/60.7M [00:09<00:00, 6.33MB/s]
Archive:  vk_pokemons.zip
  inflating: images/Abomasnow/0.jpg  
  inflating: images/Abomasnow/1.jpg  
  inflating: images/Abomasnow/2.jpg  
  inflating: images/Abomasnow/3.jpg  
  inflating: images/Abra/0.jpg       
  inflating: images/Abra/1.jpg       
  inflating: images/Abra/2.jpg       
  inflating: images/Abra/3.jpg       
  inflating: images/Absol/0.jpg      
  inflating: images/Absol/1.jpg      
  inflating: images/Absol/2.jpg      
  inflating: images/Absol/3.jpg      
  inflating: images/Accelgor/0.jpg   
  inflating: images/Accelgor/1.jpg   
  inflating: images/Aegislash/0.jpg  
  inflating: images/Aegislash/1.jpg 

Я пытался прикрутить BLIP-2 для генерации промптов для этих картинок, но в колаб влезает только int8, который довольно плохо справляется со своей задачей.
Поэтому я решил просто взять самый оптимальный промпт для данной задачи и прикрутить его ко всем картинкам с поправкой на каждого покемона

In [27]:
import os
import shutil
import pandas as pd
from glob import glob

root_directory = './raw_dataset/images/'
output_directory = './dataset/images/'
desc_file = 'pokedex_(Update_04.21).csv'

os.makedirs(output_directory, exist_ok=True)


In [28]:
# все все изображения в подпапках
for foldername, subfolders, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.lower().endswith('.jpg'):
            source_path = os.path.join(foldername, filename)
            target_path = os.path.join(output_directory, foldername.split(os.path.sep)[-1] + '_' + filename)
            shutil.copy(source_path, target_path)

In [30]:
# получение инфы о покемонах из этого датасета
# https://www.kaggle.com/datasets/mariotormo/complete-pokemon-dataset-updated-090420

df = pd.read_csv(desc_file)
df = df[["name", "status", "species", "type_1", "ability_1"]]
df['species'] = df['species'].apply(lambda x: x.split(' ')[0])
df

,name,status,species,type_1,ability_1
0,Bulbasaur,Normal,Seed,Grass,Overgrow
1,Ivysaur,Normal,Seed,Grass,Overgrow
2,Venusaur,Normal,Seed,Grass,Overgrow
3,Mega Venusaur,Normal,Seed,Grass,Thick Fat
4,Charmander,Normal,Lizard,Fire,Blaze
...,...,...,...,...,...
1040,Glastrier,Sub Legendary,Wild,Ice,Chilling Neigh
1041,Spectrier,Sub Legendary,Swift,Ghost,Grim Neigh
1042,Calyrex,Legendary,King,Psychic,Unnerve
1043,Calyrex Ice Rider,Legendary,High,Psychic,As One


In [31]:
# лучший промпт на котором удалось натренить
best_prompt = "highres, masterpiece, sugimori ken style, pokemon creature, without a background"
desc_prompt = ", picture of {} {} pokemon, {} element, with {} ability"

In [32]:
# сопоставление изображение-описание
imgs = glob("./dataset/images/*")
names = []
desc = []
for img in imgs:
    filename = "images/"+img.split("/")[3]
    names.append(filename)
    type2 = img.split("/")[3].split('_')[0]
    info = df.loc[df['name'] == type2]
    try:
        desc.append(best_prompt+desc_prompt.format(
            info["status"].to_list()[0],
            info["species"].to_list()[0],
            info["type_1"].to_list()[0],
            info["ability_1"].to_list()[0])
        .lower())
    except:
        desc.append(best_prompt)

In [33]:
# сохранение в .csv для чтения из datasets
metadata = pd.DataFrame()
metadata["file_name"] = names
metadata["text"] = desc
result = metadata.to_csv("./dataset/metadata.csv", index=False)

In [ ]:
!rm -rf raw_dataset/